In [2]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

Populating the interactive namespace from numpy and matplotlib


### Pull a current version of all patient tissue types

(didn't finish running this because my original file is the same size... what patients are missing?)

Starting at https://portal.gdc.cancer.gov/repository?filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.access%22%2C%22value%22%3A%5B%22open%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.data_category%22%2C%22value%22%3A%5B%22Clinical%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.data_type%22%2C%22value%22%3A%5B%22Clinical%20Supplement%22%5D%7D%7D%5D%7D <br>
and downloading the manifest

Saving the manifest as: /cellar/users/ramarty/Data/hla_ii/clinical/tissue_type/gdc_manifest.2017-12-08T18-43-48.070366.txt

In [17]:
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/clinical/tissue_type/gdc_manifest.2017-12-08T18-43-48.070366.txt', sep='\t')

In [21]:
len(df)

11211

In [18]:
def get_barcode(x):
    try:
        return x.split('.')[2][:12]
    except:
        return 'fail'
df['barcode'] = df.filename.apply(get_barcode)
#df = df[df.barcode != 'fail']
#df = df.drop_duplicates('barcode')

In [20]:
df[df.barcode == 'fail'].filename

2354    TARGET_RT_ClinicalData_Validation_20150710_pub...
2628                TARGET_AML_ClinicalData_20160714.xlsx
2896           TARGET_MDLS-NBL_ClinicalData_20151124.xlsx
3361        TARGET_CCSK_ClinicalData_20160714_public.xlsx
4169    TARGET_WT_ClinicalData_Discovery_20160714_publ...
4451    TARGET_WT_ClinicalData_Validation_20160714_pub...
6367       TARGET_OS_ClinicalData_Discovery_20150729.xlsx
7442    TARGET_RT_ClinicalData_Discovery_20150710_publ...
9039      TARGET_OS_ClinicalData_Validation_20160401.xlsx
Name: filename, dtype: object

In [14]:
len(df)

10962

In [ ]:
def create_cluster_script_clinical_download(file_ids, barcodes):

    new_script_file = '/cellar/users/ramarty/Projects/kir/scripts/cluster/TCGA.get_clinical.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(file_ids)))
        #out_file.write("#$ -t 1-10\n".format(len(file_ids)))
        out_file.write("#$ -l h_vmem=1G\n")
        out_file.write("#$ -tc 50\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set file_ids=({0})\n".format(" ".join(file_ids)))
        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("\n")

        out_file.write("set file_id=$file_ids[$SGE_TASK_ID]\n")
        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("bash /nrnb/users/ramarty/kir/scripts/GDC.clinical.sh $file_id /cellar/users/ramarty/Data/kir/TCGA/clinical/$barcode.xml\n")
        out_file.write("date\n")

In [ ]:
create_cluster_script_clinical_download(df.id, df.barcode)

In [ ]:
def get_tissue_type(barcode):
    try:
        return open('/cellar/users/ramarty/Data/kir/TCGA/clinical/{0}.xml'.format(barcode)).readlines()[1].split(':')[0][1:].upper()
    except:
        return '-'
df['tissue'] = df.barcode.apply(get_tissue_type)

In [ ]:
tissue_types_to_add = []
# need to add all of these random tissue types
clinical_dict = {}
for barcode, tissue in zip(df.barcode, df.tissue):
    #print barcode, tissue
    try:
        patient_dict = get_patient_clinical_information(barcode, tissue)
        patient_dict['tissue'] = tissue
        clinical_dict[barcode] = patient_dict
    except:
        print barcode, tissue

In [ ]:
clinical_df = pd.DataFrame(clinical_dict).transpose()

In [ ]:
clinical_df.to_csv('/cellar/users/ramarty/Data/kir/TCGA/clinical_df.csv')

## Can I merge HLA tissue types with current one to get a more complete list?

In [22]:
current_tissue = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_tissues.csv', index_col=0)

In [27]:
old_tissue = pd.read_csv('/cellar/users/ramarty/Data/hla/sandbox/david/patient_tissues.csv')

In [24]:
len(current_tissue), len(old_tissue)

(10959, 9176)

In [30]:
current_tissue['Sample'] = current_tissue.index
current_tissue.head()

,Tissue,Sample
TCGA-02-0001,GBM,TCGA-02-0001
TCGA-02-0003,GBM,TCGA-02-0003
TCGA-02-0004,GBM,TCGA-02-0004
TCGA-02-0006,GBM,TCGA-02-0006
TCGA-02-0007,GBM,TCGA-02-0007


In [29]:
old_tissue.head()

,Sample,Tissue
0,TCGA-3H-AB3K,MESO
1,TCGA-3H-AB3L,MESO
2,TCGA-3H-AB3M,MESO
3,TCGA-3H-AB3O,MESO
4,TCGA-3H-AB3S,MESO


In [35]:
df = pd.concat([current_tissue, old_tissue]).drop_duplicates().reset_index()

In [38]:
len(df)

11175

In [39]:
df[['Sample', 'Tissue']].to_csv('/cellar/users/ramarty/Data/hla_ii/clinical/tissue_type/patient_tissues.csv')